In [160]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from datetime import datetime, timedelta
import calendar
import holidays
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [162]:
#viz setup
# sns.set(style='whitegrid',font_scale=1.75,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))
sns.set(style='whitegrid',font_scale=1.5,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))

In [163]:
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
regions = ["_ENC","_MAC","_MTN","_NEC","_PAC","PUS","_WNC","_WSC","_ESC","_SAC"]
sectors = ['RES','COM']

In [164]:
idx = 0
for state in states:
    for sector in sectors:
        response_consumption = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.SALES.{}-{}.M".format(state,sector))
        j_consumption = response_consumption.json()
        tmp_consumption = pd.DataFrame(j_consumption['series'][0]['data'],columns=['month','sales_mkwh'])
        tmp_consumption['state'] = state
        tmp_consumption['sector'] = sector
        
        response_consumers = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.CUSTOMERS.{}-{}.M".format(state,sector))
        j_consumers = response_consumers.json()
        tmp_consumers = pd.DataFrame(j_consumers['series'][0]['data'],columns=['month','consumers'])
        tmp_consumers['state'] = state
        tmp_consumers['sector'] = sector
        
        response_price = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.PRICE.{}-{}.M".format(state,sector))
        j_price = response_price.json()
        tmp_price = pd.DataFrame(j_price['series'][0]['data'],columns=['month','price'])
        tmp_price['state'] = state
        tmp_price['sector'] = sector
        
        tmp = tmp_consumption.merge(tmp_consumers,how='left',on=['month','state','sector']).merge(tmp_price,how='left',on=['month','state','sector'])
        
        if idx == 0:
            energy_data = tmp.copy()
        else:
            energy_data = energy_data.append(tmp)
        idx = idx +1

In [165]:
idx = 0
for region in regions:
    response_cool = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWCD{}.M".format(region))
    j_cool = response_cool.json()
    tmp_cool = pd.DataFrame(j_cool['series'][0]['data'],columns=['month','cooling_days'])
    tmp_cool['region'] = region
    
    response_heat = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWHD{}.M".format(region))
    j_heat = response_heat.json()
    tmp_heat = pd.DataFrame(j_heat['series'][0]['data'],columns=['month','heating_days'])
    tmp_heat['region'] = region
    
    tmp = tmp_cool.merge(tmp_heat,how='left',on=['month','region'])
    if idx == 0:
        heating_cooling_days = tmp.copy()
    else:
        heating_cooling_days = heating_cooling_days.append(tmp)
    idx = idx +1

In [166]:
energy_data['revenue'] = energy_data.sales_mkwh*energy_data.price
country = energy_data.groupby(['month','sector']).sum().reset_index()
country['state'] = 'USA'
country.price = country.revenue/country.sales_mkwh

In [167]:
energy_data = energy_data.append(country)

In [168]:
energy_data['use_per_capita'] = energy_data.sales_mkwh*1000000/energy_data.consumers

In [169]:
heating_cooling_days.region = [re.sub('_','',r) for r in heating_cooling_days.region]

In [170]:
states.extend(['USA'])

In [171]:
state_region_mapping = pd.DataFrame(data={'state': states})

In [172]:
state_region_mapping['region'] = ''

In [173]:
state_region_mapping.loc[state_region_mapping.state.isin(['WA','OR','CA']),'region'] = 'PAC'
state_region_mapping.loc[state_region_mapping.state.isin(['MT','ID','WY','NV','UT','CO','AZ','NM']),'region'] = 'MTN'
state_region_mapping.loc[state_region_mapping.state.isin(['ND','SD','MN','NE','IA','KS','MO']),'region'] = 'WNC'
state_region_mapping.loc[state_region_mapping.state.isin(['OK','TX','AR','LA']),'region'] = 'WSC'
state_region_mapping.loc[state_region_mapping.state.isin(['WI','IL','IN','MI','OH']),'region'] = 'ENC'
state_region_mapping.loc[state_region_mapping.state.isin(['KY','TN','MS','AL']),'region'] = 'ESC'
state_region_mapping.loc[state_region_mapping.state.isin(['WV','MD','DE','VA','NC','SC','GA','FL','DC']),'region'] = 'SAC'
state_region_mapping.loc[state_region_mapping.state.isin(['NY','PA','NJ']),'region'] = 'MAC'
state_region_mapping.loc[state_region_mapping.state.isin(['RI','CT','MA','NH','VT','ME']),'region'] = 'NEC'
state_region_mapping.loc[state_region_mapping.state.isin(['USA']),'region'] = 'PUS'

In [174]:
energy_data = energy_data.merge(state_region_mapping,how='left',on='state')

In [175]:
energy_data= energy_data.merge(heating_cooling_days,how='left',on=['month','region'])

In [176]:
energy_data = energy_data.dropna()

In [177]:
energy_data = pd.concat([energy_data,pd.get_dummies(energy_data.sector)],axis=1)

In [178]:
energy_data['time'] = [12*(int(d[0:4])-2008)+int(d[4:6]) for d in energy_data.month]

In [179]:
energy_data['year'] = [int(d[0:4]) for d in energy_data.month]
energy_data['mon'] = [int(d[4:6]) for d in energy_data.month]

In [180]:
def get_season(m):
    if (m == 12)|(m<=2):
        return 'winter'
    if (m>=3)&(m<=5):
        return 'spring'
    if(m>=6)&(m<=8):
        return 'summer'
    if(m>=9)&(m<=11):
        return 'fall'

In [181]:
energy_data['season'] = energy_data.mon.apply(get_season)

In [182]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,heating_days,COM,RES,time,year,mon,season
0,202010,2137.29371,AL,RES,2286425.0,13.07,27934.428790,934.775341,ESC,66.799124,154.607615,0,1,154,2020,10,fall
1,202009,2875.42861,AL,RES,2325509.0,13.18,37898.149080,1236.472794,ESC,211.894717,19.453204,0,1,153,2020,9,fall
2,202008,3569.57496,AL,RES,2322835.0,12.76,45547.776490,1536.732036,ESC,388.011338,0.000000,0,1,152,2020,8,summer
3,202007,3587.72858,AL,RES,2319406.0,12.69,45528.275680,1546.830775,ESC,462.496476,0.000000,0,1,151,2020,7,summer
4,202006,2912.11315,AL,RES,2314926.0,12.93,37653.623029,1257.972458,ESC,297.090326,2.700401,0,1,150,2020,6,summer


In [183]:
energy_data['date'] = [datetime(y,m,1) for y,m in zip(energy_data.year, energy_data.mon)]

In [184]:
def get_datetime_features(date):
    st = date
    en = date + relativedelta(months=1) - relativedelta(days=1)
    
    ## number of days in month
    num_days = len(pd.date_range(st,en))
    ## number of weekends in month
    num_weekends = pd.date_range(st,en).weekday.isin([5,6]).sum()
    ## number of holidays in month
    us_holidays = holidays.US(years=date.year)
    us_holidays = pd.DataFrame(us_holidays.items(),columns=['date','hol'])
    us_holidays['date'] = pd.to_datetime(us_holidays.date)
    num_holidays = len(us_holidays[(us_holidays.date.dt.month == date.month) & ~(us_holidays.date.dt.weekday.isin([5,6]))])
    
    num_weekends_or_holidays = num_holidays+num_weekends
    ## % of weekdays in month
    pct_weekdays = 1 - (num_holidays+num_weekends)/num_days
    
    return num_days, num_weekends_or_holidays, pct_weekdays

In [185]:
energy_data['num_days'], energy_data['num_hols'], energy_data['pct_weekdays'] = zip(*energy_data.date.apply(get_datetime_features)) 

In [186]:
energy_data['y'] = energy_data.use_per_capita/energy_data.num_days

In [187]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
0,202010,2137.29371,AL,RES,2286425.0,13.07,27934.428790,934.775341,ESC,66.799124,...,1,154,2020,10,fall,2020-10-01,31,10,0.677419,30.154043
1,202009,2875.42861,AL,RES,2325509.0,13.18,37898.149080,1236.472794,ESC,211.894717,...,1,153,2020,9,fall,2020-09-01,30,9,0.700000,41.215760
2,202008,3569.57496,AL,RES,2322835.0,12.76,45547.776490,1536.732036,ESC,388.011338,...,1,152,2020,8,summer,2020-08-01,31,10,0.677419,49.572001
3,202007,3587.72858,AL,RES,2319406.0,12.69,45528.275680,1546.830775,ESC,462.496476,...,1,151,2020,7,summer,2020-07-01,31,9,0.709677,49.897767
4,202006,2912.11315,AL,RES,2314926.0,12.93,37653.623029,1257.972458,ESC,297.090326,...,1,150,2020,6,summer,2020-06-01,30,8,0.733333,41.932415


In [188]:
energy_data.to_csv('energy/energy_data.csv',index=False)